# XGBoost

In [1]:
from datasets import get_datasets
datasets = get_datasets()

In [2]:
import xgboost as xgb
data = datasets.small.resplit(0.75,0,0.25)

In [3]:
dtrain = xgb.DMatrix(data = data.train_split.X, label=data.train_split.y)
dtest = xgb.DMatrix(data = data.test_split.X, label=data.train_split.y)

c:\users\martin\documents\skole\pyenv\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [4]:
clf = xgb.XGBClassifier(missing=9999999999,
                max_depth = 7,
                n_estimators=700,
                learning_rate=0.1, 
                nthread=4,
                subsample=1.0,
                colsample_bytree=0.5,
                min_child_weight = 3,
                seed=1301)

In [5]:
xgb_param = clf.get_xgb_params()

In [ ]:
#do cross validation
print ('Start cross validation')
cvresult = xgb.cv(xgb_param, dtrain, num_boost_round=5000, nfold=10, metrics=['auc'],
     early_stopping_rounds=50,verbose_eval=10, stratified=True, seed=1301)
print('Best number of trees = {}'.format(cvresult.shape[0]))
clf.set_params(n_estimators=cvresult.shape[0])
print('Fit on the trainingsdata')
clf.fit(X_sel, y, eval_metric='auc')

In [ ]:
print('Overall AUC:', roc_auc_score(y, clf.predict_proba(X_sel)[:,1]))
print('Predict the probabilities based on features in the test set')
pred = clf.predict_proba(sel_test, ntree_limit=cvresult.shape[0])

## Attempt number two

In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold


X = data.train_split.X
y = data.train_split.y


bayes_cv_tuner = BayesSearchCV(estimator = xgb.XGBClassifier(
                                n_jobs = -1,
                                objective = 'binary:logistic',
                                eval_metric = 'auc',
                                learning_rate = 0.1,
                                silent=1,
                                early_stopping = 200,
                                n_estimators = 8000,
                                tree_method='approx'),
    search_spaces = {
        'min_child_weight': (15, 20),
        'max_depth': (6, 8),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-2, 1000, 'log-uniform'),
        'reg_alpha': (1e-2, 1.0, 'log-uniform'),
        'gamma': (1e-2, 0.5, 'log-uniform'),
        'min_child_weight': (0, 20),
        'scale_pos_weight': (1e-6, 500, 'log-uniform')
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42),
    n_jobs = 3,
    n_iter = 10,   
    verbose = 500,
    refit = True,
    random_state = 786)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name + "_cv_results.csv")
    
result = bayes_cv_tuner.fit(X.values, y.values, callback=status_print)

TypeError: __init__() got an unexpected keyword argument 'fit_params'